In [21]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': '../sst5/train_sst5.csv', 'val' : '../sst5/val_sst5.csv', 'test': '../sst5/test_sst5.csv'})

Using custom data configuration default-1ec86d73e728f420
Reusing dataset csv (C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
train_ds, val_ds, test_ds = dataset["train"], dataset["val"], dataset["test"]

In [23]:
train_ds  

Dataset({
    features: ['sentence', 'label'],
    num_rows: 8544
})

In [24]:
val_ds

Dataset({
    features: ['sentence', 'label'],
    num_rows: 1101
})

In [25]:
test_ds

Dataset({
    features: ['sentence', 'label'],
    num_rows: 2210
})

In [26]:
from fastai.text.all import *
set_seed(42,True)

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\AR88750/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at C:\Use

In [28]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length = 60)
 
tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-11ef6b7551047f26.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-f932d30016af9069.arrow


  0%|          | 0/3 [00:00<?, ?ba/s]

In [30]:
import numpy as np
from datasets import load_metric

acc = load_metric("accuracy")
f1 = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    res = {"accuracy" : acc.compute(predictions=predictions, references=labels),
           "f1" : f1.compute(predictions=predictions, references=labels, average="weighted")
           }
        
    return res

In [31]:
import torch
torch.cuda.empty_cache()

In [32]:
from fastai.text.all import *
set_seed(42,True)

In [33]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length = 60)
 
tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-8c21c41cde55feed.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-2a8805f3e474ecfe.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-6ff1f394630c7c04.arrow


In [34]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments("trainer_evaluation_test", evaluation_strategy="epoch", num_train_epochs=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\AR88750/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running training *****
  Num examples = 8544
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2136


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.207000,1.130679,{'accuracy': 0.5149863760217984},{'f1': 0.48721945493637164}
2,0.888000,1.201807,{'accuracy': 0.5113533151680291},{'f1': 0.5024222546546259}


Saving model checkpoint to trainer_evaluation_test\checkpoint-500
Configuration saved in trainer_evaluation_test\checkpoint-500\config.json
Model weights saved in trainer_evaluation_test\checkpoint-500\pytorch_model.bin
Saving model checkpoint to trainer_evaluation_test\checkpoint-1000
Configuration saved in trainer_evaluation_test\checkpoint-1000\config.json
Model weights saved in trainer_evaluation_test\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 1101
  Batch size = 8
Saving model checkpoint to trainer_evaluation_test\checkpoint-1500
Configuration saved in trainer_evaluation_test\checkpoint-1500\config.json
Model weights saved in trainer_evaluation_test\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to trainer_evaluation_test\checkpoint-2000
Configuration saved in trainer_evaluatio

TrainOutput(global_step=2136, training_loss=1.0748390847824039, metrics={'train_runtime': 231.11, 'train_samples_per_second': 73.939, 'train_steps_per_second': 9.242, 'total_flos': 526894080284160.0, 'train_loss': 1.0748390847824039, 'epoch': 2.0})

In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 2210
  Batch size = 8


{'eval_loss': 1.1431002616882324,
 'eval_accuracy': {'accuracy': 0.5271493212669683},
 'eval_f1': {'f1': 0.522969080700315},
 'eval_runtime': 5.833,
 'eval_samples_per_second': 378.879,
 'eval_steps_per_second': 47.489}